# Spatial Disambiguation
This notebook documents different methods tested for spatial disambiguation. The objective is to create spatial weights that will be added to the confidence score 

Methods experimented with:

<b>[Graph-based](#graph)</b>:
- [Shortest Paths](#shortest-path)
- [Centrality](#centrality)
- [Bayesian Networks](#bayesian-networks)

<b>[Cluster-based](#cluster)</b>:
- [Density Based](#density-cluster)

In [16]:
import pandas as pd
import networkx as nx
import numpy as np
import folium

In [3]:
pd.set_option('display.min_rows', 1000)
pd.set_option('display.max_rows', 10000)

In [3]:
match = pd.read_csv("../data/matches.csv")

<span id="graph"><h1>Graph-based Algorithms</h></span>
<hr>

<span id='shortest-path'><h2>Shortest Path (Dijkstra's)</span></h>

Prototyping a graph
- requires **anchor points**: points which are confident matches.
- have to take into account records above and below anchors
- has to be iterative for the removal of duplicates.

In [6]:
# Pick a small subset of the data, anchored by 2 points
sub = match[(match.CENSUS_ID >= 326668) & (match.CENSUS_ID <= 326774)]
sub

,Unnamed: 0,CD_ID,CD_FIRST_NAME,CD_LAST_NAME,MATCH_ADDR,CD_OCCUPATION,num_matches,CENSUS_ID,CENSUS_NAMEFRSCLEAN,CENSUS_NAMELASTB,...,CENSUS_ENUMDIST,CENSUS_SEGMENT_ID,CD_BLOCK_NUM,census_occupation_listed,jaro_winkler_aggr_score,confidence_score,LONG,LAT,age_score,census_count
39506,20468,34093,Raphael,Silverstein,"29 ESSEX ST, New York, NY",tailor,1,326668,RAPHAEL,SILVERSTEIN,...,198,917,190,1,1.000,1.00,-73.989769,40.715749,1,1
39507,52853,88036,Henry,Marks,"43 ESSEX ST, New York, NY",tailor,2,326672,HENRY,MARKS,...,198,917,190,1,1.000,0.85,-73.989470,40.716326,0,1
39508,63475,104822,Harris,Levy,"44 ALLEN ST, New York, NY",pedlar,3,326688,HARRIS,LEVI,...,198,917,1267,1,0.910,0.72,-73.991798,40.716435,1,2
39509,63472,104820,Harris,Levy,"45 HESTER ST, New York, NY",grocer,3,326688,HARRIS,LEVI,...,198,917,190,1,0.910,0.72,-73.989941,40.715697,1,2
39510,63393,105156,Simon,Levi,"36 LUDLOW ST, New York, NY",tailor,4,326690,SIMON,LEVI,...,198,917,190,0,1.000,0.59,-73.990371,40.715932,0,5
39511,63397,105157,Simon,Levi,"59 HESTER ST, New York, NY",pedlar,4,326690,SIMON,LEVI,...,198,917,1268,0,1.000,0.59,-73.990716,40.715931,0,5
39512,63389,104712,Simon,Levy,"58 ORCHARD ST, New York, NY",tailor,4,326690,SIMON,LEVI,...,198,917,1268,0,0.910,0.55,-73.990787,40.716811,0,5
39513,63385,104711,Simon,Levy,"41 LUDLOW ST, New York, NY",tailor,4,326690,SIMON,LEVI,...,198,917,1268,0,0.910,0.55,-73.990485,40.716161,0,5
39514,63381,104703,Simon,Levy,"27 ESSEX ST, New York, NY",hairs,4,326690,SIMON,LEVI,...,198,917,190,0,0.910,0.55,-73.989811,40.715666,0,5
39515,63296,104620,Morris,Levy,"29 ESSEX ST, New York, NY",tailor,5,326692,MORRIS,LEVI,...,198,917,190,1,0.910,0.70,-73.989769,40.715749,1,2


Quick note: this subset was chosen as it is bounded by two anchor points (first and last record, where confidence = 1) and contains examples of census records with multiple matches, and city directory records with multiple matches. 

In [9]:
# prepare dataframe in format to create graph
# this cell creates unique ids for the nodes
sub_graph = sub.loc[:, ['CD_ID', 'CENSUS_ID', 'LONG', 'LAT', 'confidence_score', 'MATCH_ADDR']]
sub_graph['anchor'] = sub_graph['confidence_score'].apply(lambda x: 1 if x == 1 else 0)
sub_graph['node_ID'] = sub_graph.groupby('CENSUS_ID').cumcount()

letter_id = sub_graph['CENSUS_ID'].unique().tolist()
letters = [chr(x+65) for x in range(0, len(letter_id))]
letter_id = pd.DataFrame({'CENSUS_ID': letter_id, 'letter': letters})

sub_graph = sub_graph.merge(letter_id, how='left', on='CENSUS_ID', validate='many_to_one')

sub_graph['node_ID'] = sub_graph.apply(lambda row: row.letter + str(row.node_ID), axis=1)

In [30]:
sub_graph

,CD_ID,CENSUS_ID,LONG,LAT,confidence_score,MATCH_ADDR,anchor,node_ID,letter,key
0,34093,326668,-73.989769,40.715749,1.00,"29 ESSEX ST, New York, NY",1,A0,A,0
1,88036,326672,-73.989470,40.716326,0.85,"43 ESSEX ST, New York, NY",0,B0,B,0
2,104822,326688,-73.991798,40.716435,0.72,"44 ALLEN ST, New York, NY",0,C0,C,0
3,104820,326688,-73.989941,40.715697,0.72,"45 HESTER ST, New York, NY",0,C1,C,0
4,105156,326690,-73.990371,40.715932,0.59,"36 LUDLOW ST, New York, NY",0,D0,D,0
5,105157,326690,-73.990716,40.715931,0.59,"59 HESTER ST, New York, NY",0,D1,D,0
6,104712,326690,-73.990787,40.716811,0.55,"58 ORCHARD ST, New York, NY",0,D2,D,0
7,104711,326690,-73.990485,40.716161,0.55,"41 LUDLOW ST, New York, NY",0,D3,D,0
8,104703,326690,-73.989811,40.715666,0.55,"27 ESSEX ST, New York, NY",0,D4,D,0
9,104620,326692,-73.989769,40.715749,0.70,"29 ESSEX ST, New York, NY",0,E0,E,0


In [21]:
# produce table where each row is an edge on the graph
g = sub_graph
g['key'] = 0
g = g.merge(g, on='key')
g['key'] = g.apply(lambda row: 1 if ord(row.letter_x) - ord(row.letter_y) == -1 else 0, axis = 1)
g = g[g.key == 1]

## weight is currently ONLY manhattan distance - need to ultimately add confidence score
g['weight'] = g.apply(lambda row: ((row.LONG_y - row.LONG_x)**2 + (row.LAT_y - row.LONG_x)**2)**(1/2), axis=1)

In [22]:
g_edges = [(row.node_ID_x, row.node_ID_y, row.weight) for index, row in g.iterrows()]

INSERT A CONCEPTUAL DIAGRAM to show what the graph looks like + what least cost is doing

In [50]:
# create graph and use algorithm to find shortest path
graph = nx.DiGraph()
graph.add_weighted_edges_from(g_edges)
path = nx.dijkstra_path(graph, 'A0', 'I0')

In [51]:
# these nodes form the shortest-path (i.e. closest to each other)
path

['A0', 'B0', 'C1', 'D4', 'E0', 'F0', 'G0', 'H0', 'I0']

In [48]:
## SIMPLE VISUALIZATION: NEEDS IMPROVEMENT.
import matplotlib.pyplot as plt

pos = nx.spring_layout(graph)
nx.draw(graph,pos,node_color='k',with_labels=True)
# draw path in red
path_edges = list(zip(path,path[1:]))
nx.draw_networkx_nodes(graph,pos,nodelist=path,node_color='r')
nx.draw_networkx_edges(graph,pos,edgelist=path_edges,edge_color='r',width=5)
plt.axis('equal')
plt.show()

C:\Users\jolen\Anaconda3\lib\site-packages\networkx\drawing\nx_pylab.py:579: MatplotlibDeprecationWarning: 
The iterable function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use np.iterable instead.
  if not cb.iterable(width):


<Figure size 640x480 with 1 Axes>

In [24]:
import folium

map_viz = folium.Map(
    location = (sub_graph.LAT.tolist()[0], sub_graph.LONG.tolist()[0]),
    zoom_start = 30,
)

pal = {'A': 'red', 'B': 'orange', 'C': 'yellow', 'D': 'green', 'E': 'blue', 'F': 'purple', 'G': 'white', 'H': 'black', 'I': 'pink', 'J': 'brown'}

for index, row in sub_graph.iterrows():
    folium.CircleMarker(
        location = (row['LAT'], row['LONG']),
        opacity = 0,
        fill = True,
        fill_opacity = 0.7,
        color = pal[row['letter']],
        size = 0.1,
        tooltip = ('<b>Node</b>: ' + str(row['node_ID']) + '<br>'
                   '<b>Address:</b> ' + row['MATCH_ADDR'] + '<br>')
    ).add_to(map_viz)

points = [(row.LAT, row.LONG) for index, row in sub_graph[sub_graph.node_ID.isin(path)].iterrows()]

folium.PolyLine(points).add_to(map_viz)

In [25]:
# HOW TO READ THIS MAP:
# same color = these points are different location options for the same census record
# lines connect the shortest path
# hover over to see details

map_viz

### What is happening?
Some nodes 'disappeared' because they are overlapping - this may or may not be a problem. 
- It could be a problem because the nodes may not ACTUALLY be unique, ie the same CD record is a potential for many census records and was selected multiple time in the least cost algorithm, although we know that a CD record only maps to one census record.
- Not be a problem - different CD records may share the same address.  

### Pros of this method
- Identifies shortest path which in most cases will turn out to be the combination we want to select
- This ensures ONLY one match is returned for each census record

### Some issues/improvements:
- Could integrate confidence score into weighting process
- How to convert this knowledge into a spatial weight? Just +1 confidence to the nodes in the shortest path?
- How to deal with census records that may truly not have a CD match? Set threshold of weight? (How to identify in the first place?)
    - Note: this is VERY important to figure out because outliers can actually impact the graph negatively. In the figure below, C0 is probably the correct match but because of an outlier that has only one match, C1 is chosen instead.
    
<img src="./img/hnyc_graph_problem_eg.png">

<hr>

<span id="centrality"><h2>Centrality</h2></span>

This adds on to the shortest path algorithm; it generates an 'importance' score for each node based on the centrality score. There are [many centrality measures](https://towardsdatascience.com/graph-analytics-introduction-and-concepts-of-centrality-8f5543b55de3), here we experiment with Closeness Centrality and Betweenness Centrality.

### Closeness Centrality
- Closeness Centrality of a node is the sum of the shortest path distance from it to all other nodes
- This basically incorporates the notion of a cluster (how well the node fits with its other options), while still remaining that the most central node for a certain 
- Note that in the networkx, the centrality is returned as its inverse, so higher centrality = higher score

In [70]:
# created undirected graph. otherwise, paths will only consider incoming distance.
c = nx.Graph()
c.add_weighted_edges_from(g_edges)
nx.closeness_centrality(c, distance='weight')

{'A0': 0.002030209153330192,
 'B0': 0.002600094461806535,
 'C0': 0.0032934314961088223,
 'C1': 0.0032934490240551857,
 'D0': 0.0036147536287263315,
 'D1': 0.003614751242703231,
 'D2': 0.0036147462845876207,
 'D3': 0.003614751671456342,
 'D4': 0.003614759129816838,
 'E0': 0.004234429191445897,
 'E1': 0.004234419991183584,
 'F0': 0.0036147588385825507,
 'F1': 0.0036147473642748162,
 'F2': 0.00361474596151563,
 'G0': 0.0031532955970880487,
 'H0': 0.002511949010133608,
 'I0': 0.002030203499590536,
 'J0': 0.0016652214340582}

### Betweenness Centrality
Here, we experiment a modification to the betweenness centrality algorithm.   
**Betweenness centrality of a node**: how often the node appears in the shortest path between 2 nodes.
- Typically, all node pairs are considered when calculating centrality, however networkx presents a modification where one can provide a subset of start and end nodes. In our case, this is basically the same as calculating shortest path:

In [72]:
# A0 and J0 not selected since the calculation excludes the node itself. 
# For us, it's not a problem as we already know A0 and J0 are anchors
nx.betweenness_centrality_subset(graph, ['A0'], ['J0'], weight='weight')

{'A0': 0.0,
 'B0': 1.0,
 'C0': 0.0,
 'C1': 1.0,
 'D0': 0.0,
 'D1': 0.0,
 'D2': 0.0,
 'D3': 0.0,
 'D4': 1.0,
 'E0': 1.0,
 'E1': 0.0,
 'F0': 1.0,
 'F1': 0.0,
 'F2': 0.0,
 'G0': 1.0,
 'H0': 1.0,
 'I0': 1.0,
 'J0': 0.0}

One modification to this could be to instead find how many times a node appears in the $k$ shortest paths, which is more appropriate in our case. Then, the centrality for a node ($N_i$) is:

$$\sum \frac{(no. of paths including N_i)}{k} $$

** This really just a simple modification of the current implementation of [between centrality subset](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.betweenness_centrality_subset.html#networkx.algorithms.centrality.betweenness_centrality_subset)

In [75]:
# this lists all possible paths in the graph, sorted by weight (shortest to longest)
all_paths = nx.shortest_simple_paths(graph, 'A0', 'J0', weight='weight')
list(all_paths)

[['A0', 'B0', 'C1', 'D4', 'E0', 'F0', 'G0', 'H0', 'I0', 'J0'],
 ['A0', 'B0', 'C1', 'D0', 'E0', 'F0', 'G0', 'H0', 'I0', 'J0'],
 ['A0', 'B0', 'C1', 'D4', 'E1', 'F0', 'G0', 'H0', 'I0', 'J0'],
 ['A0', 'B0', 'C1', 'D3', 'E0', 'F0', 'G0', 'H0', 'I0', 'J0'],
 ['A0', 'B0', 'C1', 'D1', 'E0', 'F0', 'G0', 'H0', 'I0', 'J0'],
 ['A0', 'B0', 'C1', 'D4', 'E0', 'F1', 'G0', 'H0', 'I0', 'J0'],
 ['A0', 'B0', 'C1', 'D0', 'E1', 'F0', 'G0', 'H0', 'I0', 'J0'],
 ['A0', 'B0', 'C1', 'D4', 'E0', 'F2', 'G0', 'H0', 'I0', 'J0'],
 ['A0', 'B0', 'C1', 'D2', 'E0', 'F0', 'G0', 'H0', 'I0', 'J0'],
 ['A0', 'B0', 'C1', 'D3', 'E1', 'F0', 'G0', 'H0', 'I0', 'J0'],
 ['A0', 'B0', 'C1', 'D1', 'E1', 'F0', 'G0', 'H0', 'I0', 'J0'],
 ['A0', 'B0', 'C0', 'D4', 'E0', 'F0', 'G0', 'H0', 'I0', 'J0'],
 ['A0', 'B0', 'C1', 'D0', 'E0', 'F1', 'G0', 'H0', 'I0', 'J0'],
 ['A0', 'B0', 'C1', 'D0', 'E0', 'F2', 'G0', 'H0', 'I0', 'J0'],
 ['A0', 'B0', 'C1', 'D4', 'E1', 'F1', 'G0', 'H0', 'I0', 'J0'],
 ['A0', 'B0', 'C1', 'D3', 'E0', 'F1', 'G0', 'H0', 'I0',

In [110]:
## def function to achieve this:
def betweenness_centrality_k(graph, source, target, weight, k):
    # get all simple paths
    k_paths = list(nx.shortest_simple_paths(graph, source, target, weight=weight))[0:k]
    
    # initialize output: dict with nodes as keys
    output = dict.fromkeys(graph.nodes, 0)
    
    # count
    for path in k_paths:
        for node in path:
            output[node] += 1
    
    output = {key : round(value / k, 2) for key, value in output.items()}
    
    return output

In [111]:
betweenness_centrality_k(graph, 'A0', 'J0', 'weight', 10)

{'A0': 1.0,
 'B0': 1.0,
 'C0': 0.0,
 'C1': 1.0,
 'D0': 0.2,
 'D1': 0.1,
 'D2': 0.1,
 'D3': 0.2,
 'D4': 0.4,
 'E0': 0.7,
 'E1': 0.3,
 'F0': 0.8,
 'F1': 0.1,
 'F2': 0.1,
 'G0': 1.0,
 'H0': 1.0,
 'I0': 1.0,
 'J0': 1.0}

### Assessment
As we can see, both centrality measures give the same outcome in terms of nodes which participate in the shortest path for this example. However both present potential issues:

- they are still not free from the problem of outliers in the graph

For centrality, this issue is even more amplified as there is no directionality and distance includes distance to other options (e.g. C0/C1).

Moreover, for Betweenness modified:
- It is possible that a certain node shows up more in the k-shortest paths but is not actually a participant of the shortest node: what are the implications of this?
- k is a parameter that needs further tuning

However, since both algorithms are meant only to provide a weighing scheme to show relative importance between nodes for a census record (i.e. to compare between C0/C1, etc.), this may not matter greately.

<b>Further tuning option</b>:
It is possible + relatively easy to implement eigenvector centrality on top of this, but this is not done yet as I'm unsure about the mathematical validity of it (needs more research). But this is promising if we find the need to further tune this approach.

<span id="bayesian-networks"><h2>Bayesian Networks</h></span>
The principles of [bayesian networks](https://www.ics.uci.edu/~rickl/courses/cs-171/2012-wq-cs171/2012-wq-cs171-lecture-slides/2012wq171-17-BayesianNetworks.pdf) can help improve on the existing algorithm. The idea is similar to the original graph except it uses conditional probabilities as weights. 

These will only apply to census records with multiple CD records to disambiguate from.

Idea: calculate node's probability given joint probability. Issue: not sure if possible to do without conditional probabilities + need to first get joint probability.

<span id="cluster"><h1>Cluster-based Algorithms</h></span>
<hr>

Types of clustering algorithms:
- Resource: https://developers.google.com/machine-learning/clustering/clustering-algorithms
- Density-based might be useful as in this case, as compared to a centroid-based one like K-means: 
    - we are really only trying to find the best cluster (i.e. the one that contains the correct matches), which presumbly will be the densest one (see above)
    - this will also eliminate outliers
    - unlike centroid which identifies radial clusters, density-based clustering can identify linear clusters
    
A visual example of this difference:
<img src="https://miro.medium.com/max/2548/1*L-hr07E_ygPJEqDXgaoGQA.png">

<span id='density-cluster'><h2>Density</span></h>
We will use the [HDBScan Algorithm](https://www.kdnuggets.com/2020/02/understanding-density-based-clustering.html), which is a popular density-based clustering algorithm. It improves upon the standard DBScan algorithm by using a data-driven approach to determining neighborhood size.

Some extra notes and references:
- Parameter selection for HDBScan: https://hdbscan.readthedocs.io/en/latest/parameter_selection.html
    - min_cluster_size = how many points min in one cluster
    - min_samples = this algorithm looks at K-nearest neighbors to determine neighborhood size. min_samples = k
    - Because we really only want one cluster, we will allow single cluster
- Pretty good explanation of DBScan, which is what HDBScan is based on:
    - https://blog.dominodatalab.com/topology-and-density-based-clustering/

In [5]:
import hdbscan

In [42]:
cluster_sub = sub.loc[:, ['LONG', 'LAT']]
clusterer = hdbscan.HDBSCAN(min_cluster_size=10, min_samples=10, allow_single_cluster=True).fit(cluster_sub)

In [47]:
# which nodes have been selected?
nodes = []
for index, row in sub_graph.iterrows():
    if clusterer.labels_[index] == 0:
        nodes.append(row.node_ID)
nodes

['A0', 'C1', 'D0', 'D3', 'D4', 'E0', 'E1', 'F0', 'H0', 'J0']

In [43]:
map_viz = folium.Map(
    location = (sub_graph.LAT.tolist()[0], sub_graph.LONG.tolist()[0]),
    zoom_start = 30,
)

pal = {0: 'green', -1: 'black'}
    
for index, row in sub_graph.iterrows():
    folium.CircleMarker(
        location = (row['LAT'], row['LONG']),
        opacity = 0,
        fill = True,
        fill_opacity = 0.7,
        color = pal[clusterer.labels_[index]],
        size = 0.1,
        tooltip = ('<b>Node</b>: ' + str(row['node_ID']) + '<br>'
                   '<b>Address:</b> ' + row['MATCH_ADDR'] + '<br>')
    ).add_to(map_viz)
    
    
points = [(row.LAT, row.LONG) for index, row in sub_graph[sub_graph.node_ID.isin(path)].iterrows()]

folium.PolyLine(points).add_to(map_viz)

In [44]:
# green = in cluster with the anchor point
# black = not in the cluster
# lines represent path by graph algorithm
map_viz

In [31]:
# comparison of outcomes
sub_graph['graph_outcome'] = sub_graph.apply(lambda row: 1 if row.node_ID in path else 0, axis=1)
sub_graph['cluster_outcome'] = clusterer.labels_
sub_graph

,CD_ID,CENSUS_ID,LONG,LAT,confidence_score,MATCH_ADDR,anchor,node_ID,letter,key,graph_outcome,cluster_outcome
0,34093,326668,-73.989769,40.715749,1.00,"29 ESSEX ST, New York, NY",1,A0,A,0,1,0
1,88036,326672,-73.989470,40.716326,0.85,"43 ESSEX ST, New York, NY",0,B0,B,0,1,-1
2,104822,326688,-73.991798,40.716435,0.72,"44 ALLEN ST, New York, NY",0,C0,C,0,0,-1
3,104820,326688,-73.989941,40.715697,0.72,"45 HESTER ST, New York, NY",0,C1,C,0,1,0
4,105156,326690,-73.990371,40.715932,0.59,"36 LUDLOW ST, New York, NY",0,D0,D,0,0,0
5,105157,326690,-73.990716,40.715931,0.59,"59 HESTER ST, New York, NY",0,D1,D,0,0,-1
6,104712,326690,-73.990787,40.716811,0.55,"58 ORCHARD ST, New York, NY",0,D2,D,0,0,-1
7,104711,326690,-73.990485,40.716161,0.55,"41 LUDLOW ST, New York, NY",0,D3,D,0,0,0
8,104703,326690,-73.989811,40.715666,0.55,"27 ESSEX ST, New York, NY",0,D4,D,0,1,0
9,104620,326692,-73.989769,40.715749,0.70,"29 ESSEX ST, New York, NY",0,E0,E,0,1,0


### Assessment of Performance
Recap, nodes chosen: ['A0', 'C1', 'D0', 'D3', 'D4', 'E0', 'E1', 'F0', 'H0', 'J0']

There's one major issue with this:
We cannot constrain it to ensure that one point is selected for each census record. This means some census records have all matches chosen, some have none (e.g., census record D has multiple matches, B was completely left out)
- from some initial research, these seems to be the case for most clustering algorithms, as they are more for grouping points rather than 'selection' of sets (which is more catered to by graphs).

But, this could be useful in outlier detection (which indeed is something clustering algorithms are more suited for). This could be useful for setting a threshold beyond which we will not consider the match. E.g. this algorithm managed to find that B, G, and I were generally poor matches.